In [28]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use('fivethirtyeight')

#preprocessing
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag


In [2]:
import DataCleaning_reviews_5

In [3]:
df5 = DataCleaning_reviews_5.data_clean('../reviews_Grocery_and_Gourmet_Food_5.json.gz')

151254
{'reviewerID': 'A1VEELTKS8NLZB', 'asin': '616719923X', 'reviewerName': 'Amazon Customer', 'helpful': [0, 0], 'reviewText': 'Just another flavor of Kit Kat but the taste is unique and a bit different.  The only thing that is bothersome is the price.  I thought it was a bit expensive....', 'overall': 4.0, 'summary': 'Good Taste', 'unixReviewTime': 1370044800, 'reviewTime': '06 1, 2013'}
151254


In [4]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   reviewerID     151254 non-null  object 
 1   asin           151254 non-null  object 
 2   reviewerName   149761 non-null  object 
 3   reviewText     151254 non-null  object 
 4   helpful_votes  151254 non-null  object 
 5   rating         151254 non-null  float64
 6   reviewTitle    151254 non-null  object 
dtypes: float64(1), object(6)
memory usage: 9.2+ MB


In [5]:
df = df5[['reviewText','reviewTitle', 'asin']]

In [6]:
df.head()

,reviewText,reviewTitle,asin
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X
2,Really good. Great gift for any fan of green t...,Yum!,616719923X
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X


In [7]:
# check for null values
print(df['reviewText'].isnull().sum())
df['reviewTitle'].isnull().sum()  # no null values.

print(df.shape)
df.head()

0
(151254, 3)


,reviewText,reviewTitle,asin
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X
2,Really good. Great gift for any fan of green t...,Yum!,616719923X
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X


In [8]:
df["all_text"] = df["reviewText"] + ' ' + df["reviewTitle"]


<ipython-input-8-fab1430bca7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["all_text"] = df["reviewText"] + ' ' + df["reviewTitle"]


In [9]:
df.head()

,reviewText,reviewTitle,asin,all_text
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X,Just another flavor of Kit Kat but the taste i...
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X,I bought this on impulse and it comes from Jap...
2,Really good. Great gift for any fan of green t...,Yum!,616719923X,Really good. Great gift for any fan of green t...
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X,"I had never had it before, was curious to see ..."
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X,I've been looking forward to trying these afte...


In [24]:
stemmer_porter=PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
import string  
snowball_stem = SnowballStemmer(language='english')
s_words = set(stopwords.words('english'))
punc = set(string.punctuation)
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
df['cleanText1']= df['all_text'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
df['cleanText2'] = df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
df['cleanText3'] = df['cleanText2'].apply(is_only_alpha)
df['cleanText4'] = df['cleanText3'].apply(word_tokenize)
df['cleanText5'] = df['cleanText4'].apply(lambda words: [word.lower() for word in words])
df['cleanText6'] = df['cleanText5'].apply(stemmers)
df['cleanText7'] = df['cleanText6'].apply(lambda words: " ".join(words))
df = df.drop(['cleanText1','cleanText2','cleanText3','cleanText4','cleanText5','cleanText6'],axis=1)

['i', 'make', 'claims', 'sports', 'nutrition', 'or', 'nutrition', 'matter', 'expertise', 'i', 'll', 'call', 'snack', 'leave', 'place', 'exercise', 'program', 'personal', 'doctor', 'nutritionist', 'coach', 'guru', 'however', 'i', 'say', 'much', 'better', 'average', 'tasting', 'sports', 'bar', 'snack', 'name', 'preference', 'free', 'cardboad', 'ish', 'stale', 'texture', 'barely', 'identifiable', 'aftertastes', 'common', 'similar', 'products', 'while', 'noone', 'compare', 'real', 'apple', 'crisp', 'convenient', 'food', 'go', 'recognizable', 'tastes', 'oats', 'this', 'flavor', 'texture', 'predominates', 'apple', 'cinnamon', 'due', 'large', 'part', 'i', 'suspect', 'fact', 'actually', 'contains', 'ingredients', 'there', 'also', 'raisins', 'cane', 'sugars', 'including', 'fructose', 'fruit', 'juices', 'giving', 'sweet', 'cloying', 'as', 'say', 'wineries', 'taste', 'this', 'company', 'known', 'quality', 'ingredients', 'generally', 'consumer', 'friendly', 'face', 'read', 'powerbar', 'com', 'the'

In [ ]:
vectorizer = TfidfVectorizer(max_features=10_000, lowercase=False)
X = vectorizer.fit_transform(df['cleanText7'])

In [ ]:
X[10:,10:].todense()

In [151]:
# X is the tf-idf of the reviews 
# y is the target or the inputed review from the user
user_input = "I want a marinara sauce that tastes similar to how my mom makes it with fresh tomatoes, garlic, basil, and olive oil."
user_df = pd.DataFrame({'user1': [user_input]})
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
user_df['cleanText1']= user_df['user1'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
user_df['cleanText2'] = user_df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
user_df['cleanText3'] = user_df['cleanText2'].apply(is_only_alpha)
user_df['cleanText4'] = user_df['cleanText3'].apply(word_tokenize)
user_df['cleanText5'] = user_df['cleanText4'].apply(lambda words: [word.lower() for word in words])
user_df['cleanText6'] = user_df['cleanText5'].apply(stemmers)
user_df['cleanText7'] = user_df['cleanText6'].apply(lambda words: " ".join(words))
#user_df = df.drop(['cleanText1','cleanText2','cleanText3','cleanText4','cleanText5','cleanText6'],axis=1)
y = vectorizer.transform([user_df['cleanText7'][0]])

In [153]:
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(y,X)
sim_sorted= np.argsort(sim_matrix)[::-1] # idx of lowest value to highest value but with ::-1 its decending
sim_sorted

array([[75626, 87463, 36845, ..., 38110,  2959, 15071]])

In [154]:
df['cleanText7'][sim_sorted[0][-2]]

'these realli great tomato i hate metal tast can tomato tast like chanc leach tomato can t healthi these rich flavor wateri great sauc virgin oliv oil fresh basil garlic s p cook pasta firm finish cook sauc simpl delici love tomato wish i could find local least i get amazon ever without amazon no tinni metal tast'

array([[103184,  62671,  33582, ...,  74295, 117112,  70191]])

'i love to cook so i bought it for when price get to high for me to buy it ani more love it'

In [171]:
item_asin_top1 = df.iloc[sim_sorted[0][-8]].loc['asin']
item_asin_top1

'B000F3Q4AM'

In [117]:
import DataCleaning_meta

In [118]:
df_meta = DataCleaning_meta.data_clean_meta("../meta_Grocery_and_Gourmet_Food.json.gz")

287051
{'category': ['Grocery & Gourmet Food', 'Dairy, Cheese & Eggs', 'Cheese', 'Gouda'], 'tech1': '', 'description': ['BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS', 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.'], 'fit': '', 'title': 'Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs', 'also_buy': [], 'image': [], 'tech2': '', 'brand': 'Ariola Imports', 'feature': [], 'rank': '165,181 in Grocery & Gourmet Food (', 'also_view': ['B0000D9MYM', 'B0000D9MYL', 'B00ADHIGBA', 'B00H9OX598', 'B001LM42GY', 'B001LM5TDY'], 'main_cat': 'Grocery', 'similar_item': '', 'date': '', 'price': '$41.91', 'asin': '0681727810'}
287051


In [172]:
df_meta[df_meta['asin'] == item_asin_top1]

,description,brand,rank,price,asin,name,features,categories,main_category
12553,[Fresh. Fast. Easy. Gluten free. Vegetarian. W...,Scarpetta,[],$28.40,B000F3Q4AM,"Scarpetta Marinara Sauce, 19.8-Ounce Jars (Pac...",[],"[Grocery & Gourmet Food, Sauces, Gravies & Mar...",Grocery


<ipython-input-36-a24b79940a69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['all_text'].apply(is_only_alpha)


'hello my is'

0         Just another flavor of Kit Kat but the taste i...
1         I bought this on impulse and it comes from whi...
2         Really Great gift for any fan of green Just so...
3         I had never had it was curious to see what it ...
4         been looking forward to trying these after hea...
                                ...                        
151249    Delicious we tried both the regular and the bo...
151250    With the many selections of instant oatmeal ce...
151251    While I usually review CDs and as well as ente...
151252    My son and I enjoyed these oatmeal He was most...
151253    I like to eat oatmeal i the I usually buy Quak...
Name: clean_text, Length: 151254, dtype: object

In [19]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/nico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nico/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nico/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
first = df['all_text'] #save first document as first so you can play around
word_tokenize(first.lower())

AttributeError: 'Series' object has no attribute 'lower'

In [30]:
feats

['00',
 '000',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '0g',
 '0gcholesterol',
 '0gtrans',
 '0mg',
 '0mgsodium',
 '10',
 '100',
 '1000',
 '100g',
 '100gr',
 '100mg',
 '105',
 '10g',
 '10mg',
 '10oz',
 '11',
 '110',
 '1150',
 '11g',
 '12',
 '120',
 '1200',
 '125',
 '128',
 '12g',
 '12oz',
 '13',
 '130',
 '135',
 '13g',
 '14',
 '140',
 '14g',
 '15',
 '150',
 '1500',
 '155',
 '15g',
 '15mg',
 '16',
 '160',
 '160mg',
 '16g',
 '16oz',
 '17',
 '170',
 '175',
 '17g',
 '18',
 '180',
 '18g',
 '19',
 '190',
 '190mg',
 '19g',
 '1g',
 '1lb',
 '1oz',
 '1st',
 '20',
 '200',
 '2000',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '200mg',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '20g',
 '20mg',
 '20oz',
 '21',
 '210',
 '21g',
 '22',
 '220',
 '22g',
 '23',
 '230',
 '23g',
 '24',
 '240',
 '24g',
 '24oz',
 '25',
 '250',
 '25g',
 '26',
 '260',
 '27',
 '28',
 '280',
 '28g',
 '29',
 '290',
 '29g',
 '2g',
 '2gsugars',
 '2lb',
 '2nd',
 '2oz',
 '2x',
 '30',
 '300',
 '30g',

In [26]:
count_vect = CountVectorizer(lowercase=True, tokenizer=None, stop_words='english',
                             analyzer='word', max_df=1.0, min_df=1,
                             max_features=None)

target_reviews = count_vect.fit(df['all_text'].values)
target_reviews.get_feature_names()

['00',
 '000',
 '0000',
 '000000',
 '000000000001',
 '000001',
 '0001',
 '0004brita',
 '0006',
 '0008',
 '000i',
 '000mg',
 '000mgniacin',
 '000packet',
 '000s',
 '001',
 '0015',
 '0025',
 '005',
 '0069615',
 '007',
 '00703',
 '00704',
 '009373',
 '00added',
 '00am',
 '00by',
 '00cost',
 '00dark',
 '00equipment',
 '00fiber',
 '00for',
 '00found',
 '00great',
 '00mild',
 '00milk',
 '00mm',
 '00monthly',
 '00my',
 '00natural',
 '00nothing',
 '00or',
 '00overall',
 '00pm',
 '00sales',
 '00signature',
 '00something',
 '00taste',
 '00when',
 '00wonderfully',
 '01',
 '010',
 '010900005319',
 '011g',
 '013',
 '014',
 '017',
 '01ceeb65',
 '01g',
 '01hits',
 '01oz',
 '02',
 '020',
 '023',
 '024',
 '025gtotal',
 '0273',
 '02762',
 '028',
 '029',
 '03',
 '030314',
 '030613',
 '031413',
 '034',
 '03430',
 '035',
 '03510',
 '0373water',
 '03755other',
 '0377',
 '038',
 '039',
 '03979426',
 '03oz',
 '04',
 '042',
 '042512_exposing',
 '04820',
 '04and',
 '04oz',
 '05',
 '050',
 '050411',
 '0529oz',
 